## Helixscope Developer Queries

This notebook explains the metadata and back-end queries required to build the Helixscope project. 

Note ISO2 codes are used to link the `gadm28_countries` table to the summary data tables. The below mapping links the codes to human-readable names for the front-end.


*Specific warming levels*
```json
{'1.5':'1.5°C specific warming level',
 '2':'2°C specific warming level',
 '4':'4°C specific warming level',
 '6':'6°C specific warming level'}
```


*Climate Variables*

```json
{'time_perc_change_SPI48':'info',
'nbp':'info',
'time_perc_change_SPI6':'info',
'time_perc_change_SRI48':'info',
'gpp':'info',
'evap':'info',
'perc_change_low_roff':'info',
'perc_change_roff':'info',
'time_perc_change_SRI6':'info'}

```

In [1]:
import folium
import pprint
import requests
import json

## Generate Choropleths 

Country-wide choropleths based on means.

In [5]:
query = """WITH data as (SELECT * FROM master_admin0
 Where variable like 'tx'
 AND swl_info = '2')
 SELECT gadm28_countries.iso, gadm28_countries.the_geom_webmercator,
 gadm28_countries.cartodb_id, mean as mean from data inner join gadm28_countries 
 on data.iso=gadm28_countries.iso
"""

# style = """#layer {polygon-fill: ramp([mean], colorbrewer(Reds), jenks());
#                    polygon-opacity: 0.9;}"""

style = """#layer {polygon-fill: ramp([mean], colorbrewer(RdYlBu), equal(10));
                   polygon-opacity: 0.9;}"""

account = 'helixscope'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}

r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.url)
pprint.pprint(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm2 = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm2)

map_osm2

https://helixscope.carto.com/api/v1/map
{'cdn_url': {'http': 'ashbu.cartocdn.com',
             'https': 'cartocdn-ashbu.global.ssl.fastly.net',
             'templates': {'http': {'subdomains': ['0', '1', '2', '3'],
                                    'url': 'http://{s}.ashbu.cartocdn.com'},
                           'https': {'subdomains': ['a', 'b', 'c', 'd'],
                                     'url': 'https://cartocdn-ashbu_{s}.global.ssl.fastly.net'}}},
 'last_updated': '2017-10-09T10:48:23.990Z',
 'layergroupid': 'b3948ad450229db94e28884e4fdcb981:1507546103990',
 'metadata': {'analyses': [],
              'dataviews': {},
              'layers': [{'id': 'layer0',
                          'meta': {'cartocss': '#layer {polygon-fill: '
                                               '#a50026;[ mean > -35.914 ] '
                                               '{polygon-fill: #d73027;}[ mean '
                                               '> -26.688 ] {polygon-fill: '
            

## Example of getting gridded data 

In [4]:
swl_variable = '4'
climate_variable = 'tx'
data_table = 'ecearth_r3_swl_2_cl_tx_mam'

query = """WITH data as (SELECT * FROM ecearth_r3_swl_2_cl_tx_mam
 Where variable like 'tx'
 AND swl_info = '2')
 SELECT sanitized_grid.id_val, sanitized_grid.the_geom_webmercator,
 sanitized_grid.cartodb_id, mean as mean from data inner join sanitized_grid 
 on data.shape_id=sanitized_grid.id_val
"""

style = """#layer {polygon-fill: ramp([mean], colorbrewer(RdYlBu), equal(10));
                   polygon-opacity: 0.9;}"""

account = 'helixscope'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}

r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.url)
pprint.pprint(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm_grid = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm_grid)

map_osm_grid

https://helixscope.carto.com/api/v1/map
{'cdn_url': {'http': 'ashbu.cartocdn.com',
             'https': 'cartocdn-ashbu.global.ssl.fastly.net',
             'templates': {'http': {'subdomains': ['0', '1', '2', '3'],
                                    'url': 'http://{s}.ashbu.cartocdn.com'},
                           'https': {'subdomains': ['a', 'b', 'c', 'd'],
                                     'url': 'https://cartocdn-ashbu_{s}.global.ssl.fastly.net'}}},
 'last_updated': '2017-10-09T10:58:42.856Z',
 'layergroupid': 'c1cba37c8f31254bebadfe209621676d:1507546722856',
 'metadata': {'analyses': [],
              'dataviews': {},
              'layers': [{'id': 'layer0',
                          'meta': {'cartocss': '#layer {polygon-fill: '
                                               '#a50026;[ mean > '
                                               '-46.6065379753113 ] '
                                               '{polygon-fill: #d73027;}[ mean '
                             

### Example of Calling Country-wide data Tables

In [3]:
swl_variable = '4'
climate_variable = 'tx'
data_table = 'master_amin0'

query = """SELECT iso, mean, max, count, min, std, model_long_name, model_short_name, model_taxonomy
    FROM master_admin0
    Where swl_info = '4'
    AND season = 'MAM'
    AND model_short_name = 'HADGEM3'
    AND run = 1
    LIMIT 3
"""

account = 'helixscope'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": query}
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

https://helixscope.carto.com/api/v2/sql?q=SELECT+iso%2C+mean%2C+max%2C+count%2C+min%2C+std%2C+model_long_name%2C+model_short_name%2C+model_taxonomy%0A++++FROM+master_admin0%0A++++Where+swl_info+%3D+%274%27%0A++++AND+season+%3D+%27MAM%27%0A++++AND+model_short_name+%3D+%27HADGEM3%27%0A++++AND+run+%3D+1%0A++++LIMIT+3%0A
{'fields': {'count': {'type': 'number'},
            'iso': {'type': 'string'},
            'max': {'type': 'number'},
            'mean': {'type': 'number'},
            'min': {'type': 'number'},
            'model_long_name': {'type': 'string'},
            'model_short_name': {'type': 'string'},
            'model_taxonomy': {'type': 'string'},
            'std': {'type': 'number'}},
 'rows': [{'count': 382,
           'iso': 'NOR',
           'max': 14.35,
           'mean': 8.33,
           'min': 1.04,
           'model_long_name': 'HADGEM3-R1',
           'model_short_name': 'HADGEM3',
           'model_taxonomy': 'HADGEM3',
           'std': 2.71},
          {'cou

## notes + dev past this point

In [ ]:
swl_variable = '1.5'
climate_variable = 'cSoil'
data_table = 'joined_summaries_2017_08_27'

query = """WITH data as (SELECT * FROM master_admin0
 Where variable like 'cSoil'
 AND swl_info = '1.5')
 SELECT gadm28_countries.iso2, gadm28_countries.the_geom_webmercator,
 gadm28_countries.cartodb_id, mean as mean from data inner join gadm28_countries 
 on data.iso2=gadm28_countries.iso2
"""

style = """#layer {polygon-fill: ramp([mean], colorbrewer(Reds), jenks());
                   polygon-opacity: 0.9;}"""

account = 'helixscope'
urlCarto = 'https://'+account+'.carto.com/api/v1/map'
body = {
    "layers": [{
        "type": "cartodb",
        "options": {
            "sql": query,
            "cartocss":style,
            "cartocss_version": "2.1.1"
        }
    }]
}

r = requests.post(urlCarto, data=json.dumps(body), headers={'content-type': 'application/json; charset=UTF-8'})
print(r.url)
pprint.pprint(r.json())

tileUrl = 'https://'+account+'.carto.com/api/v1/map/' + r.json()['layergroupid'] + '/{z}/{x}/{y}.png32';

map_osm_test = folium.Map(location=[45.5236, 0.6750], zoom_start=3)
folium.TileLayer(
    tiles=tileUrl,
    attr='text',
    name='text',
    overlay=True
).add_to(map_osm_test)

map_osm_test

In [ ]:
import geopandas as gpd

In [ ]:
s = gpd.read_file('./data/gadm28_countries/')

In [ ]:
s.head()

In [ ]:
s[s.iso == 'SSD']

## Example of retrieving data JSON

This will be necessary to build widgets attached to the data.

In [ ]:
# example of getting the raw data

swl_variable = '1.5'
climate_variable = 'cSoil'
data_table = 'joined_summaries_2017_08_27'

query = """SELECT iso2, mean, max, count, min, std, model_long_name, model_short_name, model_taxonomy
 FROM joined_summaries_2017_08_27
 Where variable like 'cSoil'
 AND swl_info = '1.5'
"""

account = 'helixscope'
urlCarto = "https://{0}.carto.com/api/v2/sql".format(account)

sql = {"q": query}
r = requests.get(urlCarto, params=sql)
print(r.url)
pprint.pprint(r.json())

## Example of valid queries

Example of how to get the average over multiple runs of the same model, for a specific country (ES), for a specific SWL (1.5), for a specific variable (perc_change_roff).

```sql
SELECT iso2, variable, model_taxonomy, COUNT(helix_data.mean) num_runs, AVG(helix_data.mean) as model_mean, AVG(helix_data.max) as model_max
FROM helix_data
WHERE swl_info = '1.5'
AND iso2 = 'ES'
AND variable = 'perc_change_roff'
GROUP BY iso2, variable, model_taxonomy
```


Get all individual model values (multiple models and multiple runs) for a specific country, SWL, variable, returning metadata for the model taxnomy too.

```sql
SELECT iso2, model_taxonomy, run, variable, helix_data.mean as country_mean, helix_data.max as country_max
FROM helix_data
WHERE swl_info = '1.5'
AND iso2 = 'ES'
AND variable = 'perc_change_roff'
GROUP BY iso2, variable, model_taxonomy, mean, max, run
```


Get single average over all countries for a specific SWL, and specific variable (to create choropleth).

```sql
SELECT iso2, variable, AVG(helix_data.mean) as country_mean, AVG(helix_data.max) as country_max
FROM helix_data
WHERE swl_info = '1.5'
AND variable = 'cSoil'
GROUP BY iso2, variable

```

Get data average across table

```sql
SELECT iso2, variable, AVG(helix_data.mean) as model_mean, AVG(helix_data.max) as model_max
FROM joined_summaries_2017_08_27
WHERE swl_info = '1.5'
GROUP BY iso2, variable
```